# Ejemplo 007

**Problema:** Queremos obtener, para todas las celdas operativas: 
- CGI
- latitud
- longitud
- radio aproximado de celda 


#### Importamos paquetes necesarios

In [1]:
# Importación de paquetes requeridos
import os
import io
import requests
import pandas as pd
import numpy as np

from IPython.display import display, Markdown, Latex

import json

#### Funciones auxiliares

In [4]:
# Función auxiliar para visualizar los datos
def pp(obj):
    print(json.dumps(obj, indent=2))

In [5]:
# La operación queda con la forma siguiente:
# «para cada fila r, convertir el valor de la columna "col" en un tipo numérico (pd.to_numeric),
# ignorando los valores nulos (errors='coerce'), redondeando hacia abajo al entero más cercano
# (np.floor) y almacenándolo como entero de 64 bits (astype('Int64'))»

def pd_to_int(df: pd.DataFrame, col: str) -> int:
    return np.floor(pd.to_numeric(df[col], errors='coerce')).astype('Int64')

#### Parámetros del script

In [6]:
# Parmámetros
iga_api_base_url = 'http://ing-acc-movil01.personal.corp/ingenier@/symfony/public/index.php/api'

#### Test de conectividad

In [7]:
# Llamamos a un endpoint de prueba en la API de IGA para verificar conectividad
x = requests.get(iga_api_base_url + '/public/db-sync-state')
pp(x.json())

{
  "status": "success",
  "data": {
    "Uid": "71195",
    "State": "DB_SYNC_STATE_IDLE",
    "LastSyncTimestamp": "04-06-2024 10:12:42",
    "SyncState": "OK",
    "ProjectsUpdated": "DataManager, AMBA"
  }
}


#### Descarga de reportes

- Debemos descargar tres reportes: Datos de Integración, Datos Outdoor y Radios aproximados de sectores. 
- En los tres casos, los sitios que nos interesan son los on air (OA), es decir que tienen al menos una celda operativa.
- El endpoint usado es /public/report/generic-report en los tres casos.
- Los datos devueltos están en formato binario, y pueden ser leídos directamente por Pandas como un archivo .xlsx, y convertidos a un objeto DataFrame

In [13]:
# Parámetros en el payload de la request con que llamamos al endpoint de IGA:
payload = {
    # Nombre del reporte (lo completamos después)
    "reportName": None,
    # Sitios seleccionados: ninguno
    "siteNames": [],
    # Parámetros de filtrado: todos los sitios en estado operativo (OA).
    # Son los sitios que tienen al menos una celda operativa.
    "filteringParams": {"SiteVisualizationData.SiteStatus": ["OA"]},
    # Seleccionar por nombres de sitio: no
    "selectBySiteNames": False,
    # Seleccionar por parámetros de filtrado: sí
    "selectByFilteringParams": True
}

# Datos de Integración
payload["reportName"] = "datos_integracion"
x = requests.post(iga_api_base_url + '/public/report/generic-report', json=payload)
# En x almacenamos los datos devueltos por el endpoint de IGA.
# Es un archivo xlsx en formato binario

# Usando pandas, con la función read_excel, leemos los datos y los almacenamos
# en memoria como un objeto DataFrame:
with io.BytesIO(x.content) as fh:
    df_integracion_4g_raw = pd.read_excel(fh, "LTE")
    df_integracion_5g_raw = pd.read_excel(fh, "NR")

# Repetimos para los demás reportes:
payload["reportName"] = "datos_outdoor"
x = requests.post(iga_api_base_url + '/public/report/generic-report', json=payload)
with io.BytesIO(x.content) as fh:
    df_outdoor_raw = pd.read_excel(fh, "Datos outdoor")

payload["reportName"] = "datos_radio_sector"
x = requests.post(iga_api_base_url + '/public/report/generic-report', json=payload)
with io.BytesIO(x.content) as fh:
    df_radio_sector_raw = pd.read_excel(fh, "Radios aproximados de sectores")

display(df_integracion_4g_raw)
display(df_integracion_5g_raw)
display(df_outdoor_raw)
display(df_radio_sector_raw)

,Proyecto,Código,Sector,Portadora,MME,CGI,Estado,Vendor,E-NODEB Id,TAC,Physical Cell Id,Logical Cell Id,RACH Root Seq.,PA Power dBm,Ref. Signal Boosting dB,EARFCN DL,EARFCN UL,PRACH CS,Base Station Type,E-UTRAN Cell Id
0,MEDI,A01-E133,XRCESEL1G,LTE FDD_AWS_2225,MME[1_2CLI-1_2BEL-1_2MUN],722-34-2594048,Operativo,HUAWEI,10133.0,10191,402,0.0,648.0,50.8,3.00,2225.0,20225.0,11.0,BBU 3900,2594048.0
1,MEDI,A01-E133,XRCESEL1H,LTE FDD_AWS_2225,MME[1_2CLI-1_2BEL-1_2MUN],722-34-2594049,Operativo,HUAWEI,10133.0,10191,403,1.0,656.0,50.8,3.00,2225.0,20225.0,11.0,BBU 3900,2594049.0
2,MEDI,A01-E133,XRCESEL1I,LTE FDD_AWS_2225,MME[1_2CLI-1_2BEL-1_2MUN],722-34-2594050,Operativo,HUAWEI,10133.0,10191,404,2.0,664.0,50.8,3.00,2225.0,20225.0,11.0,BBU 3900,2594050.0
3,MEDI,A01-E133,XRCESEM1G,LTE FDD_1900MHz_975,MME[1_2CLI-1_2BEL-1_2MUN],722-34-2594066,No Operativo,HUAWEI,10133.0,10191,501,18.0,472.0,50.8,3.00,975.0,19075.0,11.0,BBU 3900,2594066.0
4,MEDI,A01-E133,XRCESEM1H,LTE FDD_1900MHz_975,MME[1_2CLI-1_2BEL-1_2MUN],722-34-2594067,No Operativo,HUAWEI,10133.0,10191,502,19.0,480.0,50.8,3.00,975.0,19075.0,11.0,BBU 3900,2594067.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113598,AMBA,SPU990,SBBORIN12,LTE FDD_700MHz_9360,MME[1_2CLI-1_2BEL-1_2MUN],722-34-129021444,Operativo,Huawei,503990.0,15118,25,4.0,360.0,48.4,1.77,9360.0,27360.0,12.0,BBU 5900,129021444.0
113599,AMBA,SPU990,SBBORIN13,LTE FDD_700MHz_9360,MME[1_2CLI-1_2BEL-1_2MUN],722-34-129021445,Operativo,Huawei,503990.0,15118,26,5.0,520.0,48.4,1.77,9360.0,27360.0,12.0,BBU 5900,129021445.0
113600,AMBA,SPU990,SBBORIO11,LTE FDD_850MHz_2450,MME[1_2CLI-1_2BEL-1_2MUN],722-34-129021458,Operativo,Huawei,503990.0,15118,24,18.0,290.0,48.4,1.77,2450.0,20450.0,12.0,BBU 5900,129021458.0
113601,AMBA,SPU990,SBBORIO12,LTE FDD_850MHz_2450,MME[1_2CLI-1_2BEL-1_2MUN],722-34-129021459,Operativo,Huawei,503990.0,15118,25,19.0,360.0,48.4,1.77,2450.0,20450.0,12.0,BBU 5900,129021459.0


,Proyecto,Código,Sector,Portadora,MME,CGI,Estado,Vendor,G-NODEB Id,TAC,Physical Cell Id,Logical Cell Id,RACH Root Seq.,PA Power dBm,EARFCN DL,Base Station Type,NR Cell Id
0,MEDI,A01-S006,XLCARLR11,NR_Band_n7_534000,MME[1_2CLI-1_2BEL-1_2MUN],722-34-32931938304,No Operativo,HUAWEI,2010006.0,10192,357,0.0,6,50.0,534000.0,BBU 3900,3.293194e+10
1,MEDI,A01-S006,XLCARLR12,NR_Band_n7_534000,MME[1_2CLI-1_2BEL-1_2MUN],722-34-32931938305,No Operativo,HUAWEI,2010006.0,10192,360,1.0,12,50.0,534000.0,BBU 3900,3.293194e+10
2,MEDI,A01-S006,XLCARLR13,NR_Band_n7_534000,MME[1_2CLI-1_2BEL-1_2MUN],722-34-32931938306,No Operativo,HUAWEI,2010006.0,10192,363,2.0,18,50.0,534000.0,BBU 3900,3.293194e+10
3,MEDI,A01-S021,XRCREPR11,NR_Band_n7_534000,MME[1_2CLI-1_2BEL-1_2MUN],722-34-32932184064,No Operativo,HUAWEI,2010021.0,10191,0,0.0,6,50.0,534000.0,BBU 3900,3.293218e+10
4,MEDI,A01-S021,XRCREPR12,NR_Band_n7_534000,MME[1_2CLI-1_2BEL-1_2MUN],722-34-32932184065,No Operativo,HUAWEI,2010021.0,10191,3,1.0,12,50.0,534000.0,BBU 3900,3.293218e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3587,AMBA,SPU674,SPPMTPR12,NR_Band_n7_534000,MME[1_2CLI-1_2BEL-1_2MUN],722-34-41020194817,No Operativo,Huawei,2503674.0,15118,478,1.0,54,50.0,534000.0,BBU 5900,4.102019e+10
3588,AMBA,SPU674,SPPMTPR13,NR_Band_n7_534000,MME[1_2CLI-1_2BEL-1_2MUN],722-34-41020194818,No Operativo,Huawei,2503674.0,15118,479,2.0,66,50.0,534000.0,BBU 5900,4.102019e+10
3589,AMBA,SPU816,SCPESES11,NR_Band_n78_630000,NaN,722-34-,No Operativo,NaN,NaN,15117,-1,350.0,44,50.0,630000.0,BBU 5900,NaN
3590,AMBA,SPU816,SCPESES12,NR_Band_n78_630000,NaN,722-34-,No Operativo,NaN,NaN,15117,-1,351.0,0,50.0,630000.0,BBU 5900,NaN


,Proyecto,Código,Sector,Portadora,Estado,Latitud Antena,Longitud Antena,Id Antena,Modelo Antena,Tipo Antena,...,Azimuth Físico,Azimuth Efectivo,Tilt Mec.,Tilt Eléc.,Twist,Ancho Horiz.,Ancho Vert.,Ganancia dBi,Cota de Terreno,MIMO Mode (Netchart)
0,MEDI,A01-E133,XRCESEL1G,LTE FDD_AWS_2225,Operativo,-33.142181,-64.348739,1,AAU3940,Legacy,...,40.0,39.0,-0.0,2.0,0.0,73,12,13.972812,436.86,(Sin datos)
1,MEDI,A01-E133,XRCESEL1H,LTE FDD_AWS_2225,Operativo,-33.142181,-64.348739,2,AAU3940,Legacy,...,160.0,159.0,-0.0,2.0,0.0,73,12,13.972812,436.86,(Sin datos)
2,MEDI,A01-E133,XRCESEL1I,LTE FDD_AWS_2225,Operativo,-33.142181,-64.348739,3,AAU3940,Legacy,...,280.0,279.0,-0.0,2.0,0.0,73,12,13.972812,436.86,(Sin datos)
3,MEDI,A01-E133,XRCESEM1G,LTE FDD_1900MHz_975,No Operativo,-33.142181,-64.348739,1,AAU3940,Legacy,...,40.0,39.0,-0.0,2.0,0.0,73,12,13.972812,436.86,(Sin datos)
4,MEDI,A01-E133,XRCESEM1H,LTE FDD_1900MHz_975,No Operativo,-33.142181,-64.348739,2,AAU3940,Legacy,...,160.0,159.0,-0.0,2.0,0.0,73,12,13.972812,436.86,(Sin datos)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166897,AMBA,SPU990,SBBORIU12,WCDMA1900_9788,No Operativo,-34.790932,-58.239595,2,DBXDH-6565A-VTM,Legacy,...,180.0,180.0,-0.0,2.0,0.0,64,10,15.331015,18.77,NaN
166898,AMBA,SPU990,SBBORIU13,WCDMA1900_9788,No Operativo,-34.790932,-58.239595,3,DBXDH-6565A-VTM,Legacy,...,300.0,300.0,-0.0,10.0,0.0,64,10,15.331015,18.77,NaN
166899,AMBA,SPU990,SBBORIV21,WCDMA850_4358,No Operativo,-34.790932,-58.239595,1,DBXDH-6565A-VTM,Legacy,...,60.0,60.0,-0.0,4.0,0.0,64,10,15.331015,18.77,NaN
166900,AMBA,SPU990,SBBORIV22,WCDMA850_4358,No Operativo,-34.790932,-58.239595,2,DBXDH-6565A-VTM,Legacy,...,180.0,180.0,-0.0,4.0,0.0,64,10,15.331015,18.77,NaN


,Código Sitio,EMG,Estado Sitio,Tipo Estación,Sector,Tecnol./Banda,Estado Sector,Radio Aprox. Sector [m]
0,A01-E133,XRCESE,OA,EasyM,XRCESEL1G,LTE FDD_AWS_2225,Operativo,618.326588
1,A01-E133,XRCESE,OA,EasyM,XRCESEL1H,LTE FDD_AWS_2225,Operativo,573.343038
2,A01-E133,XRCESE,OA,EasyM,XRCESEL1I,LTE FDD_AWS_2225,Operativo,713.040137
3,A01-E133,XRCESE,OA,EasyM,XRCESEM1G,LTE FDD_1900MHz_975,No Operativo,618.326588
4,A01-E133,XRCESE,OA,EasyM,XRCESEM1H,LTE FDD_1900MHz_975,No Operativo,573.343038
...,...,...,...,...,...,...,...,...
166355,SPU990,SBBORI,OA,Macro,SBBORIU12,WCDMA1900_9788,No Operativo,4492.913208
166356,SPU990,SBBORI,OA,Macro,SBBORIU13,WCDMA1900_9788,No Operativo,2678.886356
166357,SPU990,SBBORI,OA,Macro,SBBORIV21,WCDMA850_4358,No Operativo,3991.228227
166358,SPU990,SBBORI,OA,Macro,SBBORIV22,WCDMA850_4358,No Operativo,3991.228227


#### Procesamiento de los datos

- Filtramos las celdas y nos quedamos únicamente con las que estén en estado operativo
- De los datos Outdoor, obtenemos para cada celda las coordenadas
- De los datos de Integración, obtenemos para cada celda el CGI
- De los datos de Radio, obtenemos para cada celda el radio aproximado
- Unimos las tres tablas usando el campo nombre de celda

In [35]:
df_1 = (
    df_outdoor_raw.loc[df_outdoor_raw['Estado'] == 'Operativo']
    [['Código', 'Sector', 'Estado', 'Latitud Antena', 'Longitud Antena']]
).set_index('Sector')
display(df_1)

df_2 = pd.concat([
        df_integracion_4g_raw[df_integracion_4g_raw['Estado'] == 'Operativo']
        [['Sector', 'CGI']],
        df_integracion_5g_raw[df_integracion_5g_raw['Estado'] == 'Operativo']
        [['Sector', 'CGI']],
    ], axis=0).set_index('Sector')
display(df_2)

df_3 = (
    df_radio_sector_raw[df_radio_sector_raw['Estado Sector'] == 'Operativo']
    [['Sector', 'Radio Aprox. Sector [m]']]
).set_index('Sector')
display(df_3)

,Código,Estado,Latitud Antena,Longitud Antena
Sector,,,,
XRCESEL1G,A01-E133,Operativo,-33.142181,-64.348739
XRCESEL1H,A01-E133,Operativo,-33.142181,-64.348739
XRCESEL1I,A01-E133,Operativo,-33.142181,-64.348739
XRCEALL1G,A01-E134,Operativo,-33.141588,-64.338766
XRCEALL1H,A01-E134,Operativo,-33.141588,-64.338766
...,...,...,...,...
SBBORIN12,SPU990,Operativo,-34.790932,-58.239595
SBBORIN13,SPU990,Operativo,-34.790932,-58.239595
SBBORIO11,SPU990,Operativo,-34.790932,-58.239595


,CGI
Sector,
XRCESEL1G,722-34-2594048
XRCESEL1H,722-34-2594049
XRCESEL1I,722-34-2594050
XRCEALL1G,722-34-2594051
XRCEALL1H,722-34-2594052
...,...
SNXBSJS12,722-34-41174712671
SNXBSJS13,722-34-41174712672
SN10TPS11,722-34-41174958430


,Radio Aprox. Sector [m]
Sector,
XRCESEL1G,618.326588
XRCESEL1H,573.343038
XRCESEL1I,713.040137
XRCEALL1G,673.497619
XRCEALL1H,1133.431630
...,...
SBBORIN12,1140.790140
SBBORIN13,1507.300350
SBBORIO11,1910.961364


In [38]:
df = df_1.join(df_2, how='inner').join(df_3, how='inner')
display(df)

,Código,Estado,Latitud Antena,Longitud Antena,CGI,Radio Aprox. Sector [m]
Sector,,,,,,
29JUTPL1M,CI_089,Operativo,-34.604167,-58.381111,722-34-129046784,50.000000
29JUTPL1N,CI_089,Operativo,-34.604167,-58.381111,722-34-129046785,50.000000
29JUTPL1O,CI_089,Operativo,-34.604167,-58.381111,722-34-129046786,50.000000
29JUTPL1P,CI_089,Operativo,-34.604167,-58.381111,722-34-129046790,50.000000
29JUTPM1M,CI_089,Operativo,-34.604167,-58.381111,722-34-129046787,50.000000
...,...,...,...,...,...,...
ZZAVABN12,A04-S427,Operativo,-33.015297,-60.865048,722-34-10349316,7588.505275
ZZAVABN13,A04-S427,Operativo,-33.015297,-60.865048,722-34-10349317,7588.505275
ZZAVABO11,A04-S427,Operativo,-33.015297,-60.865048,722-34-10349330,6932.486230


In [40]:
# Creamos la carpeta 'output' si no existe
os.makedirs('output', exist_ok=True)

# Exportamos los datos como .csv
df.to_csv('output/ejemplo-007.csv', index=True)